In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

load model

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/clean_Helcaraxe/Elenwe_Iobs_model")

Change the last layer from using a sigmoid function to use a softmax function. For this the last layer needs to be removed and replaced.

In [ ]:
#Define input layer
input = model.input
#Remove the last layer
model.layers.pop()
#Define output layer
class_layer = tf.keras.layers.Dense(2, activation='softmax', name="class_layer")
output = class_layer(model.layers[-1].output)
#build new models with new output layer
class_model = tf.keras.models.Model(inputs=input, outputs=[output])
#compile new model
OPTI = tf.keras.optimizers.Nadam(learning_rate=0.00001)
class_model.compile(optimizer=OPTI,
              loss="binary_crossentropy")

New model has to be trained very brief to connect the new layer to the old model. Therefore plots and models are prepared like in model_trainer.ipynb

In [ ]:
#Get data for short and low retrain
plots = np.load("/content/drive/MyDrive/clean_Helcaraxe/helcaraxe_elenwe_binplots_train.npy")
labels = np.load("/content/drive/MyDrive/clean_Helcaraxe/helcaraxe_elenwe_train_labels.npy", allow_pickle=True)
#Preprocess histograms
resh_plots = plots.reshape(len(plots),80,80,1)
resh_plots = tf.image.per_image_standardization(resh_plots)
fin_plots = np.asarray(resh_plots).astype(np.float32)
labels = np.asarray(labels).astype(np.float32)
labels = tf.keras.utils.to_categorical(labels, num_classes=2)
fin_plots = tf.convert_to_tensor(fin_plots)
#Split sets
train_size_lim = int((len(fin_plots)*0.8))
train_plot =  fin_plots[:train_size_lim]
val_plot = fin_plots[train_size_lim:]
train_label = labels[:train_size_lim]
val_label = labels[train_size_lim:]

In [ ]:
#Training preparations
import os
import datetime
early_stopping_cb = keras.callbacks.EarlyStopping(patience=12, restore_best_weights=True, monitor='val_loss', mode="min")

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/My Drive/Ice-ring",
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
#Train
history = class_model.fit(train_plot, train_label,
                    epochs=1,
                    validation_data=(val_plot, val_label),
                    callbacks=[early_stopping_cb, tensorboard_cb, model_checkpoint_cb]
                    )

330/330 [==============================] - 52s 58ms/step - loss: 0.6856 - val_loss: 0.6900


save model

In [ ]:
model.save("/content/drive/MyDrive/clean_Helcaraxe/class_model_elenwe_Iobs")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/clean_Helcaraxe/class_model_elenwe_Iobs/assets
